读取CIFAR-10数据

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# 数据预处理
transform_train = transforms.Compose([
    # 随机裁剪
    transforms.RandomCrop(32, padding=4),
    # 随机水平翻转
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # 归一化
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


ResNet-18 

In [3]:
import torch.nn as nn
import torch.optim as optim

# 定义BasicBlock
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        out = nn.ReLU()(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return nn.ReLU()(out)

# 定义模型
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        self.linear = nn.Linear(512, 10)

    def _make_layer(self, in_planes, out_planes, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = nn.AvgPool2d(4)(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

训练测试代码

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练并测试ResNet-18

In [5]:
net = ResNet18()
net.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, net, criterion, optimizer)
    test_loop(testloader, net, criterion)
    scheduler.step()
print("Done!")

Epoch 1
-------------------------------
loss: 2.310217  [   64/50000]
loss: 2.143452  [ 6464/50000]
loss: 1.893483  [12864/50000]
loss: 2.029742  [19264/50000]
loss: 1.751530  [25664/50000]
loss: 1.682641  [32064/50000]
loss: 1.808219  [38464/50000]
loss: 1.640991  [44864/50000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 1.510957 

Epoch 2
-------------------------------
loss: 1.486124  [   64/50000]
loss: 1.436495  [ 6464/50000]
loss: 1.304010  [12864/50000]
loss: 1.362925  [19264/50000]
loss: 1.464761  [25664/50000]
loss: 1.342310  [32064/50000]
loss: 1.360921  [38464/50000]
loss: 1.386345  [44864/50000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.280816 

Epoch 3
-------------------------------
loss: 1.220246  [   64/50000]
loss: 1.226170  [ 6464/50000]
loss: 1.128777  [12864/50000]
loss: 0.995255  [19264/50000]
loss: 1.030885  [25664/50000]
loss: 1.011014  [32064/50000]
loss: 0.995390  [38464/50000]
loss: 1.135408  [44864/50000]
Test Error: 
 Accuracy: 51.4%, Avg loss: 1.428640 

Epoc